In [1]:
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor
librispeech_dummy = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

model = EncodecModel.from_pretrained("facebook/encodec_24khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")
librispeech_dummy = librispeech_dummy.cast_column("audio", Audio(sampling_rate=processor.sampling_rate))
audio_sample = librispeech_dummy[-1]["audio"]["array"]
inputs = processor(raw_audio=audio_sample, sampling_rate=processor.sampling_rate, return_tensors="pt")

encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])
audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]
# or the equivalent with a forward pass
audio_values = model(inputs["input_values"], inputs["padding_mask"]).audio_values

c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for hf-internal-testing/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hf-internal-testing/librispeech_asr_dummy
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\josed\Documents\AudioLM\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecat

In [3]:
inputs["input_values"].shape

torch.Size([1, 1, 107520])

In [4]:
import torch
torch.rand(1,1,150000)

tensor([[[0.1410, 0.8580, 0.8115,  ..., 0.9627, 0.3983, 0.5188]]])

In [41]:
from audiolm.data_preparation import AudioDataLoader
import os
dataloader = AudioDataLoader(os.getcwd() + "\\..\\data\\datasets",sample_frequency=24000)

In [42]:
next(iter(dataloader)).shape

torch.Size([2, 1, 72000])

In [43]:
audio = (next(iter(dataloader))[0:1,: , :])
audio.shape

torch.Size([1, 1, 72000])

In [44]:
en = model.encode(audio)

In [45]:
en["audio_codes"].shape

torch.Size([1, 1, 2, 225])

In [46]:
model.decode(en["audio_codes"],en["audio_scales"])["audio_values"]

tensor([[[ 2.7902e-04, -2.5119e-04,  4.1721e-05,  ...,  2.2882e-04,
           2.2978e-04,  2.2582e-04]]], grad_fn=<ConvolutionBackward0>)

In [47]:
import torchaudio

decoded = model.decode(en["audio_codes"],en["audio_scales"])["audio_values"].detach().squeeze(0)

torchaudio.save("prova.flac",decoded,24000)